# Imports 

In [10]:
import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Load dataset

In [11]:
df = pd.read_csv(r"D:\Python\SmartSignalAI\data\An Urban Multi-Operator QoE-Aware Dataset for Cell\raw_dataset.csv")
print("Dataset loaded successfully!")

Dataset loaded successfully!


# Feature Engineering - Distance Calculation

In [12]:
# Calculate the geographical distance between:
# - User device location
# - Cell tower (Node) location
# Using the Haversine formula

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c

# Create Distance Feature

In [13]:
df['Distance_km'] = haversine(
    df['Latitude'],
    df['Longitude'],
    df['Node_Latitude'],
    df['Node_Longitude']
)

print("Distance feature created successfully!")

Distance feature created successfully!


# Feature & Target Selection

In [15]:
# Selected features for RSRP prediction:
# - Distance_km
# - Speed

# Target:
# - Level (RSRP)

# Continuous features
cont_features = ['Distance_km', 'Speed', 'ElapsedTime']

# Categorical features (need encoding)
cat_features = ['Operatorname', 'NetworkTech', 'Mobility']

# Target
y = df['Level']  # RSRP

# Combine continuous + categorical
X = df[cont_features + cat_features]

print("Feature matrix shape:", X.shape)
print("Target shape:", y.shape)

Feature matrix shape: (30925, 6)
Target shape: (30925,)


# Train / Validation / Test Split

In [16]:
# Train-Validation-Test Split

# First split: Train + Temp (70% train, 30% temp)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42
)

# Second split: Validation + Test (50% validation, 50% test of the temp set)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    random_state=42
)

print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)
print("Test shape:", X_test.shape)

Train shape: (21647, 6)
Validation shape: (4639, 6)
Test shape: (4639, 6)


# Outlier Handling

In [17]:
# Data Cleaning: Remove outliers based on RSRP values
# Outlier handling (y_train clipping -50 to -140)
# Filtering out only “bad” target rows
# Features corresponding to those rows are removed automatically

mask = (y_train >= -140) & (y_train <= -50)

X_train = X_train[mask]
y_train = y_train[mask]

# Validation = simulate unseen data

# Test = simulate real-world deployment

# Real world WILL contain noise

# If you clean val/test, you are artificially making your model look better.

# Feauture Scaling (NO leakage)

In [19]:
# Feature Scaling - Standardization
# Scaling & Encoding in One Step with ColumnTransformer
# Feature Scaling: Standardize features to have mean=0 and std=1

# ColumnTransformer to apply StandardScaler to continuous features and OneHotEncoder to categorical features in one step

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), cont_features),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), cat_features)
    ]
)

# Fit on training data only
X_train_processed = preprocessor.fit_transform(X_train)

# Transform validation and test
X_val_processed   = preprocessor.transform(X_val)
X_test_processed  = preprocessor.transform(X_test)

print("Preprocessing (scaling + encoding) completed successfully!")

Preprocessing (scaling + encoding) completed successfully!


# Saving Processed Data

In [20]:
# Save preprocessed data and scaler for future use

joblib.dump(X_train_processed, r"D:\Python\SmartSignalAI\data\X_train.pkl") # Preprocessed train features & target
joblib.dump(y_train, r"D:\Python\SmartSignalAI\data\y_train.pkl")

joblib.dump(X_val_processed, r"D:\Python\SmartSignalAI\data\X_val.pkl")    # Preprocessed validation data
joblib.dump(y_val, r"D:\Python\SmartSignalAI\data\y_val.pkl")

joblib.dump(X_test_processed, r"D:\Python\SmartSignalAI\data\X_test.pkl")  # Preprocessed test data
joblib.dump(y_test, r"D:\Python\SmartSignalAI\data\y_test.pkl")

joblib.dump(preprocessor, r"D:\Python\SmartSignalAI\data\preprocessor.pkl")

print("All processed datasets saved successfully!")

All processed datasets saved successfully!
